In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Import dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cobra

from src.gem.yeast8model import Yeast8Model
from src.calc.ablation import get_ablation_ratio
from src.calc.flux import compare_fluxes
from src.calc.growth import get_exch_saturation
from src.viz.bar import ablation_barplot, compare_ablation_times

# Construct models of cells of interest, optimise

In [ ]:
glc_exch_rate = 16.89

wt = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.solution = wt.optimize()

BY4741 = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
BY4741.make_auxotroph("BY4741")
BY4741.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
BY4741.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
BY4741.solution = BY4741.optimize()

BY4742 = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
BY4742.make_auxotroph("BY4742")
BY4742.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
BY4742.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
BY4742.solution = BY4742.optimize()

zwf1 = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
zwf1.make_auxotroph("BY4741")
zwf1.knock_out_list(["YNL241C"])
zwf1.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
zwf1.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
zwf1.solution = zwf1.optimize()

tsa2 = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
tsa2.make_auxotroph("BY4742")
tsa2.knock_out_list(["YDR453C"])
tsa2.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
tsa2.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
tsa2.solution = tsa2.optimize()

# Ablate

TODO: For loop all this?  It's repetitive.

## Wild type

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
ablation_barplot(wt.ablation_result, ax)
ax.set_title('Wild type')
plt.show()

ratio = get_ablation_ratio(wt.ablation_result)
print(ratio)

## BY4741

In [ ]:
BY4741.ablation_result = BY4741.ablate()

fig, ax = plt.subplots()
ablation_barplot(BY4741.ablation_result, ax)
ax.set_title('BY4741 (in supplemented media)')
plt.show()

ratio = get_ablation_ratio(BY4741.ablation_result)
print(ratio)

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt.ablation_result, BY4741.ablation_result, ax_fc)

## BY4742

In [ ]:
BY4742.ablation_result = BY4742.ablate()

fig, ax = plt.subplots()
ablation_barplot(BY4742.ablation_result, ax)
ax.set_title('BY4742 (in supplemented media)')
plt.show()

ratio = get_ablation_ratio(BY4742.ablation_result)
print(ratio)

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt.ablation_result, BY4742.ablation_result, ax_fc)

In [ ]:
zwf1.ablation_result = zwf1.ablate()
fig, ax = plt.subplots()
ablation_barplot(zwf1.ablation_result, ax)
ax.set_title('zwf1$\Delta$ BY4741 (in supplemented media)')
plt.show()

ratio = get_ablation_ratio(zwf1.ablation_result)
print(ratio)

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(BY4741.ablation_result, zwf1.ablation_result, ax_fc)

In [ ]:
tsa2.ablation_result = tsa2.ablate()
fig, ax = plt.subplots()
ablation_barplot(tsa2.ablation_result, ax)
ax.set_title('tsa2$\Delta$ BY4742 (in supplemented media)')
plt.show()

ratio = get_ablation_ratio(tsa2.ablation_result)
print(ratio)

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(BY4742.ablation_result, tsa2.ablation_result, ax_fc)

In [ ]:
wt.ablation_result

In [ ]:
BY4741.ablation_result

In [ ]:
BY4742.ablation_result

In [ ]:
zwf1.ablation_result

In [ ]:
tsa2.ablation_result

In [ ]:
wt_ablation_result_orig = wt.ablation_result.copy()

## Add amino acids to minimal media

In [ ]:
amino_exch_list = [
    'r_1873', 'r_1879', 'r_1880', 'r_1881', 'r_1883',
    'r_1889', 'r_1891', 'r_1810', 'r_1893', 'r_1897',
    'r_1899', 'r_1900', 'r_1902', 'r_1903', 'r_1904',
    'r_1906', 'r_1911', 'r_1912', 'r_1913', 'r_1914',
]

wt.reset_to_file()
wt.unrestrict_growth()
wt.add_media_components(amino_exch_list)
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

In [ ]:
for reaction_id in wt.model.medium.keys():
    print(wt.model.reactions.get_by_id(reaction_id).name)

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
ablation_barplot(wt.ablation_result, ax)
ax.set_title('Wild type, with all amino acids supplemented')
plt.show()

ratio = get_ablation_ratio(wt.ablation_result)
print(ratio)

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

> You can see that it takes less time to make proteins.

## Add nucleotides to minimal media

In [ ]:
nucl_exch_list = [
    'r_1639', 'r_1705', 'r_1818', 'r_2090'
]

wt.reset_to_file()
wt.unrestrict_growth()
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.add_media_components(nucl_exch_list)

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
ablation_barplot(wt.ablation_result, ax)
ax.set_title('Wild type, with all NTPs supplemented')
plt.show()

ratio = get_ablation_ratio(wt.ablation_result)
print(ratio)

In [ ]:
wt.ablation_result

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

## Add deoxyribonucleotides to media

In [ ]:
deoxnucl_exch_list = [
    'r_1643', 'r_1702', 'r_1820', 'r_2073'
]

wt.reset_to_file()
wt.unrestrict_growth()
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.add_media_components(deoxnucl_exch_list)

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
ablation_barplot(wt.ablation_result, ax)
ax.set_title('Wild type, with all dNTPs supplemented')
plt.show()

ratio = get_ablation_ratio(wt.ablation_result)
print(ratio)

In [ ]:
wt.ablation_result

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

## Carbon sources

Pyruvate

In [ ]:
wt.reset_to_file()
wt.unrestrict_growth()
wt.add_media_components(['r_2033'])
# Set uptake rate (under saturation) to get a growth rate consistent with experiments.
wt.model.reactions.get_by_id('r_2033').bounds = (-5.476, 0)
wt.remove_media_components(['r_1714', 'r_1714_REV'])
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
ablation_barplot(wt.ablation_result, ax)
ax.set_title('Wild type, with pyruvate as carbon source')
plt.show()

ratio = get_ablation_ratio(wt.ablation_result)
print(ratio)

In [ ]:
wt.ablation_result

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

Glucose limitation

In [ ]:
glc_exch_rate = 1.329
# Simulates 10 mg/L by assuming a 3-hour doubling time
# and using the saturation curve at the end of this notebook

wt.reset_to_file()
wt.unrestrict_growth()
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
ablation_barplot(wt.ablation_result, ax)
ax.set_title('Wild type, with glucose limitation')
plt.show()

ratio = get_ablation_ratio(wt.ablation_result)
print(ratio)

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

# Compare fluxes

In [ ]:
diff_fluxes_sorted = compare_fluxes(BY4741, zwf1)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
diff_fluxes_sorted = compare_fluxes(BY4741, tsa2)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, BY4741)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
for reaction_id in BY4741.model.medium.keys():
    print(BY4741.model.reactions.get_by_id(reaction_id).name)

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, BY4741)

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, wt)

In [ ]:
diff_fluxes_sorted

# Effect of carbon source uptake on growth rate

## Glucose

Optimise wt with glucose unconstrained.

In [ ]:
wt.reset_to_file()
wt.unrestrict_growth()
wt.solution = wt.optimize()

Get saturated glucose uptake:

In [ ]:
wt.solution['r_1714_REV']

> You should get 16.9 mmol g<sub>DW</sub><sup>-1</sup> h<sup>-1</sup>.  This agrees with Elsemman et al. (2022): they predict saturation at 18.6 mmol g<sub>DW</sub><sup>-1</sup> h<sup>-1</sup> and report a range of 16 ~ 19 mmol g<sub>DW</sub><sup>-1</sup> h<sup>-1</sup> from the literature (Blank et al., 2004).

Sweep, across all strains.

In [ ]:
wt_suppl = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
wt_suppl.add_media_components([
    "r_1893",
    "r_1899",
    "r_1902",
    "r_2090",
    "r_1893_REV",
    "r_1899_REV",
    "r_1902_REV",
    "r_2090_REV",
])

In [ ]:
# positive values because i want it increasing in a plot
glc_exch_rates = np.linspace(0, 18.6, 100)

ymodels = [wt, wt_suppl, BY4741, zwf1, tsa2]
ymodel_labels = [
    'prototroph (wild type)',
    'prototroph, in BY4741 supplemented media',
    'BY4741, in supplemented media',
    'BY4741 zwf1, in supplemented media',
    'BY4742 tsa2, in supplemented media',
]

In [ ]:
ymodel_growthrates = [
    get_exch_saturation(ymodel, "r_1714", glc_exch_rates)
    for ymodel in ymodels
]
print('optimisations done')

In [ ]:
fig, ax = plt.subplots()
for ymodel_growthrate, ymodel_label in zip(ymodel_growthrates, ymodel_labels):
    ax.plot(glc_exch_rates, ymodel_growthrate, label=ymodel_label)
ax.set_xlim((0,20))
ax.set_xlabel('Glucose exchange rate (mmol/gDW/h)')
ax.set_ylabel('Growth rate (/h)')
ax.set_title('Effect of glucose exchange rate on growth rate')
ax.legend()

> Results from wild-type are similar to with Elsemman et al. (2022), figure 2B. Saturation (or near it) seems to be reached at ~8.45 mmol/gDW/h.

Get glucose exchange rate that results in a certain growth rate...

This is based on linear interpolation.  The implied function takes growth rate as the independent value and glucose exchange rate as the dependent value (i.e. the inverse function of the saturation curve).

If necessary, get growth rate from doubling time.  Based on: $T = \frac{\log 2}{r}$, where $T$ is the doubling time and $r$ is the growth rate.

In [ ]:
# Get growth rate from doubling time
dt = 3
np.log(2)/dt

In [ ]:
# Get exchange rate
wt_growthrate = ymodel_growthrates[0]
np.interp(0.3808579645, wt_growthrate, glc_exch_rates)

## Pyruvate

In [ ]:
wt.reset_to_file()
wt.unrestrict_growth()
wt.add_media_components(['r_2033'])
wt.remove_media_components(['r_1714', 'r_1714_REV'])
wt.solution = wt.optimize()

Saturated uptake

In [ ]:
wt.solution['r_2033']

> Should get: 8.46

Sweep

In [ ]:
pyr_exch_rates = np.linspace(0, 19.2, 100)
ymodel_growthrates = [
    get_exch_saturation(ymodel, "r_2033", pyr_exch_rates)
    for ymodel in ymodels
]
print('optimisations done')

In [ ]:
fig, ax = plt.subplots()
for ymodel_growthrate, ymodel_label in zip(ymodel_growthrates, ymodel_labels):
    ax.plot(pyr_exch_rates, ymodel_growthrate, label=ymodel_label)
ax.set_xlim((0,10))
ax.set_xlabel('Pyruvate exchange rate (mmol/gDW/h)')
ax.set_ylabel('Growth rate (/h)')
ax.set_title('Effect of pyruvate exchange rate on growth rate')
ax.legend()

> Should get saturation at 4.27

In [ ]:
# Get exchange rate
wt_growthrate = ymodel_growthrates[0]
np.interp(0.253616, wt_growthrate, pyr_exch_rates)

# Effect of nitrogen source uptake on growth rate

## Ammonium

In [ ]:
wt.reset_to_file()
wt.unrestrict_growth()
wt.add_media_components(['r_1654'])
wt.solution = wt.optimize()

Saturated uptake

In [ ]:
wt.solution['r_1654']

> Should get 2.88

Sweep.

Note: not removing glucose or else growth doesn't happen at all.

In [ ]:
nh4_exch_rates = np.linspace(0, 3.0, 100)
ymodel_growthrates = [
    get_exch_saturation(ymodel, "r_1654", nh4_exch_rates, remove_glucose=False)
    for ymodel in ymodels
]
print('optimisations done')

In [ ]:
fig, ax = plt.subplots()
for ymodel_growthrate, ymodel_label in zip(ymodel_growthrates, ymodel_labels):
    ax.plot(nh4_exch_rates, ymodel_growthrate, label=ymodel_label)
ax.set_xlim((0,3))
ax.set_xlabel('Ammonium exchange rate (mmol/gDW/h)')
ax.set_ylabel('Growth rate (/h)')
ax.set_title('Effect of ammonium exchange rate on growth rate')
ax.legend()

> Saturation for wild type should be 1.45

In [ ]:
# Get exchange rate
wt_growthrate = ymodel_growthrates[0]
np.interp(0.38089, wt_growthrate, nh4_exch_rates)